# Using the MNIST data compressed with JPEG method. 



#Load Libraries

In [1]:
from pennylane import numpy as np
import cv2
import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt 
import math

2023-11-30 00:51:30.069118: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 00:51:30.071662: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 00:51:30.108045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 00:51:30.108074: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 00:51:30.108089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
def get_jpeg_data(img, out_length=50):
  # Load the image

  # Convert the image to YCrCb color space
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


  # Define the desired size for the subsampled image
  new_width = img.shape[0]//2  # Set to half of the original width
  new_height = img.shape[1]//2  # Set to half of the original height

  # Resize the image to the desired size for subsampling
  subsampled_image = cv2.resize(img, (new_width, new_height))

  # Perform JPEG encoding
  retval, buf = cv2.imencode('.jpg', subsampled_image, [cv2.IMWRITE_JPEG_QUALITY, 10])

  def find_matching_slice(byte_list, target_slice):
      for i in range(len(byte_list) - len(target_slice) + 1):
          if byte_list[i:i + len(target_slice)] == target_slice:
              return i
      return -1

  byte_list = buf.tolist()

  # Define the start and end sequences to search for
  start_sequence = [0xFF, 0xDA]
  end_sequence = [0xFF, 0xD9]

  # Find the starting and ending indices
  start_index = find_matching_slice(byte_list, start_sequence)
  end_index = find_matching_slice(byte_list, end_sequence)

  # Extract the data between start and end sequences
  if start_index != -1 and end_index != -1:
      extracted_data = byte_list[start_index + len(start_sequence):end_index]
      # Convert the extracted data back to a NumPy array if needed
      extracted_data_np = np.array(extracted_data, dtype=np.uint8)

      # Pad to fixed size
      if extracted_data_np.shape[0] < out_length:
        pad_width = out_length - extracted_data_np.shape[0]
        extracted_data_np = np.pad(extracted_data_np, (0, pad_width), mode='constant', constant_values=0)
      if extracted_data_np.shape[0] > out_length:
         extracted_data_np = extracted_data_np[:out_length]
      # print("Extracted data:", extracted_data_np)
      # print("Data Length: ", len(extracted_data_np))
  else:
      print("Start and end sequences not found in the data.")
  return extracted_data_np



# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Resize/subsample images 
sub_dim = 16

train_images_subsampled = np.zeros((train_images.shape[0], sub_dim, sub_dim))
test_images_subsampled = np.zeros((test_images.shape[0], sub_dim, sub_dim))

for i in range(train_images.shape[0]):
    train_images_subsampled[i] = cv2.resize(train_images[i], (sub_dim, sub_dim))
for i in range(test_images.shape[0]):
    test_images_subsampled[i] = cv2.resize(test_images[i], (sub_dim, sub_dim))

    
print('train_images_subsampled.shape: ', train_images_subsampled.shape)
print('test_images_subsampled.shape: ', test_images_subsampled.shape)

# Filter a subset of digits and a fixed number of samples
digits = [0, 1]
train_subset = 500
test_subset = 200

train_filter = np.isin(train_labels, digits)
train_images = train_images_subsampled[train_filter][:train_subset]
train_labels = train_labels[train_filter][:train_subset]

test_filter = np.isin(test_labels, digits)
test_images = test_images_subsampled[test_filter][:test_subset]
test_labels = test_labels[test_filter][:test_subset]


# Convert to JPEG embedding
len_compressed_data = 20
train_images_jpeg = np.zeros((train_images.shape[0], len_compressed_data))
for i in range(len(train_images)):
    train_images_jpeg[i] = get_jpeg_data(train_images[i], len_compressed_data)

test_images_jpeg = np.zeros((test_images.shape[0], len_compressed_data))
for i in range(len(test_images)):
    test_images_jpeg[i] = get_jpeg_data(test_images[i], len_compressed_data)
    



train_images_subsampled.shape:  (60000, 16, 16)
test_images_subsampled.shape:  (10000, 16, 16)


In [3]:
%%time
def Model1():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([

        keras.layers.Dense(30),
        keras.layers.Dense(15),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

c_model = Model1()

c_history = c_model.fit(
    train_images_jpeg,
    train_labels,
    validation_split=0.2,
    batch_size=16,
    epochs=30,
    verbose=2,
)

# Evaluate the model on the test data to compute accuracy
loss, accuracy = c_model.evaluate(test_images_jpeg, test_labels)

# Print the accuracy
print(f'\n\nSimple ANN Test accuracy: {accuracy:.4f}')

Epoch 1/30


2023-11-30 00:51:32.104509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-30 00:51:32.147670: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


25/25 - 1s - loss: 80.2220 - accuracy: 0.2375 - val_loss: 23.7518 - val_accuracy: 0.5400 - 533ms/epoch - 21ms/step
Epoch 2/30
25/25 - 0s - loss: 13.2368 - accuracy: 0.7350 - val_loss: 12.9365 - val_accuracy: 0.7300 - 63ms/epoch - 3ms/step
Epoch 3/30
25/25 - 0s - loss: 8.4912 - accuracy: 0.8025 - val_loss: 10.6619 - val_accuracy: 0.7500 - 62ms/epoch - 2ms/step
Epoch 4/30
25/25 - 0s - loss: 7.2697 - accuracy: 0.7850 - val_loss: 8.9280 - val_accuracy: 0.7700 - 59ms/epoch - 2ms/step
Epoch 5/30
25/25 - 0s - loss: 5.2276 - accuracy: 0.8150 - val_loss: 7.1793 - val_accuracy: 0.7600 - 52ms/epoch - 2ms/step
Epoch 6/30
25/25 - 0s - loss: 4.4739 - accuracy: 0.7750 - val_loss: 5.7513 - val_accuracy: 0.7500 - 63ms/epoch - 3ms/step
Epoch 7/30
25/25 - 0s - loss: 3.4225 - accuracy: 0.7675 - val_loss: 5.9504 - val_accuracy: 0.7100 - 52ms/epoch - 2ms/step
Epoch 8/30
25/25 - 0s - loss: 2.9878 - accuracy: 0.7675 - val_loss: 3.8150 - val_accuracy: 0.7200 - 51ms/epoch - 2ms/step
Epoch 9/30
25/25 - 0s - loss

# Quantum Classifier

In [4]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import math

import matplotlib.pyplot as plt
%matplotlib inline


def reshape_x(input_array):
    # Calculate the number of subarrays needed
    num_subarrays = len(input_array) // 3 + (len(input_array) % 3 != 0)

    # Split the array into subarrays of size 3
    subarrays = [input_array[i * 3:(i + 1) * 3] for i in range(num_subarrays)]

    # Check if the last subarray has fewer than 3 elements and pad with zeros if necessary
    if len(subarrays[-1]) < 3:
        subarrays[-1] = np.pad(subarrays[-1], (0, 3 - len(subarrays[-1])), mode='constant')
        
    return subarrays


def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x[i], dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

def test(params, x, y, state_labels=None):
    """
    Tests on a given set of data.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        predicted (array([int]): predicted labels for test data
        output_states (array[float]): output quantum states from the circuit
    """
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = [fidel_function(dm) for dm in dm_labels]
        best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)

def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T

label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = np.array([label_0, label_1], requires_grad=False)


def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]
        

In [5]:
def train_q_classifier(num_layers = 10, stepsize = 0.1, momentum = 0.9, epochs = 10, batch_size = 32, n_qubits=1):

    X_train = train_images_jpeg
    y_train = train_labels
    X_test = test_images_jpeg
    y_test = test_labels
    
    

    opt = NesterovMomentumOptimizer(stepsize=stepsize, momentum=momentum)

    # initialize random weights
    params = np.random.uniform(size=(num_layers, 2*n_qubits, X_train.shape[1]), requires_grad=True)

    predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)

    predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)

    # save predictions with random weights for comparison
    initial_predictions = predicted_test

    loss = cost(params, X_test, y_test, state_labels)

    print(
        "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
            0, loss, accuracy_train, accuracy_test
        )
    )

    for it in range(epochs):
        for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
            params, _, _, _ = opt.step(cost, params, Xbatch, ybatch, state_labels)

        predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
        accuracy_train = accuracy_score(y_train, predicted_train)
        loss = cost(params, X_train, y_train, state_labels)

        predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
        accuracy_test = accuracy_score(y_test, predicted_test)
        res = [it + 1, loss, accuracy_train, accuracy_test]
        print(
            "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
                *res
            )
        )
        
    return params, float(accuracy_test)

# 1 Qubit

In [7]:
dev = qml.device("lightning.gpu", wires=1)


@qml.qnode(dev, interface="autograd")
def qcircuit(params, x, y):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for l in range(params.shape[0]): # Iterate num_layers times
        
        w_0 = params[l, 0]
        b_0 = params[l, 1, 0]
        
        
        # Compact Encoding 
        for w, x in zip(reshape_x(w_0), reshape_x(x)):
            encoding_0 = np.dot(w, x) + b_0
            qml.Rot(encoding_0, encoding_0, encoding_0, wires=0)
            
    return qml.expval(qml.Hermitian(y, wires=[0]))


In [8]:
%%time
params, accuracy = train_q_classifier(num_layers=5, stepsize=0.2, epochs=5)
print(f'Accuracy of 1 qubit model with 5 layer: {accuracy}')

Epoch:  0 | Cost: 0.427853 | Train accuracy: 0.422000 | Test Accuracy: 0.345000
Epoch:  1 | Loss: 0.312766 | Train accuracy: 0.546000 | Test accuracy: 0.455000
Epoch:  2 | Loss: 0.274649 | Train accuracy: 0.620000 | Test accuracy: 0.695000
Epoch:  3 | Loss: 0.350124 | Train accuracy: 0.488000 | Test accuracy: 0.465000
Epoch:  4 | Loss: 0.287211 | Train accuracy: 0.548000 | Test accuracy: 0.465000
Epoch:  5 | Loss: 0.328605 | Train accuracy: 0.508000 | Test accuracy: 0.505000
Accuracy of 1 qubit model with 5 layer: 0.505
CPU times: user 1min 57s, sys: 30.6 ms, total: 1min 57s
Wall time: 2min 7s


# 2 Qubits

In [9]:
dev = qml.device("lightning.gpu", wires=2)


@qml.qnode(dev, interface="autograd")
def qcircuit(params, x, y):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for l in range(params.shape[0]): # Iterate num_layers times
        
        w_0 = params[l, 0]
        b_0 = params[l, 1, 0]
        w_1 = params[l, 2]
        b_1 = params[l, 3, 0]
        
        # Compact Encoding 
        for w, x in zip(reshape_x(w_0), reshape_x(x)):
            encoding_0 = np.dot(w, x) + b_0
            qml.Rot(encoding_0, encoding_0, encoding_0, wires=0)
        
        for w, x in zip(reshape_x(w_1), reshape_x(x)):
            encoding_1 = np.dot(w, x) + b_1
            qml.Rot(encoding_1, encoding_1, encoding_1, wires=0)
            
    return qml.expval(qml.Hermitian(y, wires=[0]))


In [10]:
%%time
params, accuracy = train_q_classifier(num_layers=5, stepsize=0.2, epochs=5, n_qubits=2)
print(f'Accuracy of 2 qubit model with 5 layer: {accuracy}')

Epoch:  0 | Cost: 0.300306 | Train accuracy: 0.526000 | Test Accuracy: 0.470000
Epoch:  1 | Loss: 0.398670 | Train accuracy: 0.384000 | Test accuracy: 0.395000
Epoch:  2 | Loss: 0.392303 | Train accuracy: 0.386000 | Test accuracy: 0.410000
Epoch:  3 | Loss: 0.308709 | Train accuracy: 0.526000 | Test accuracy: 0.545000
Epoch:  4 | Loss: 0.364614 | Train accuracy: 0.410000 | Test accuracy: 0.495000
Epoch:  5 | Loss: 0.415796 | Train accuracy: 0.392000 | Test accuracy: 0.320000
Accuracy of 2 qubit model with 5 layer: 0.32
CPU times: user 2min 47s, sys: 92.6 ms, total: 2min 47s
Wall time: 3min 1s


# 4 Qubits

In [11]:
dev = qml.device("lightning.gpu", wires=4)


@qml.qnode(dev, interface="autograd")
def qcircuit(params, x, y):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for l in range(params.shape[0]): # Iterate num_layers times
        
        w_0 = params[l, 0]
        b_0 = params[l, 1, 0]
        w_1 = params[l, 2]
        b_1 = params[l, 3, 0]
        w_2 = params[l, 4]
        b_2 = params[l, 5, 0]
        w_3 = params[l, 6]
        b_3 = params[l, 7, 0]
        
        # Compact Encoding 
        for w, x in zip(reshape_x(w_0), reshape_x(x)):
            encoding_0 = np.dot(w, x) + b_0
            qml.Rot(encoding_0, encoding_0, encoding_0, wires=0)
        
        for w, x in zip(reshape_x(w_1), reshape_x(x)):
            encoding_1 = np.dot(w, x) + b_1
            qml.Rot(encoding_1, encoding_1, encoding_1, wires=0)
        
        for w, x in zip(reshape_x(w_2), reshape_x(x)):
            encoding_2 = np.dot(w, x) + b_2
            qml.Rot(encoding_2, encoding_2, encoding_2, wires=0)
        
        for w, x in zip(reshape_x(w_3), reshape_x(x)):
            encoding_3 = np.dot(w, x) + b_3
            qml.Rot(encoding_3, encoding_3, encoding_3, wires=0)
            
    return qml.expval(qml.Hermitian(y, wires=[0]))


In [12]:
%%time
params, accuracy = train_q_classifier(num_layers=5, stepsize=0.2, epochs=5, n_qubits=4)
print(f'Accuracy of 4 qubit model with 5 layer: {accuracy}')

Epoch:  0 | Cost: 0.323262 | Train accuracy: 0.450000 | Test Accuracy: 0.390000
Epoch:  1 | Loss: 0.444846 | Train accuracy: 0.384000 | Test accuracy: 0.390000
Epoch:  2 | Loss: 0.300855 | Train accuracy: 0.570000 | Test accuracy: 0.615000
Epoch:  3 | Loss: 0.286739 | Train accuracy: 0.552000 | Test accuracy: 0.535000
Epoch:  4 | Loss: 0.368826 | Train accuracy: 0.458000 | Test accuracy: 0.360000
Epoch:  5 | Loss: 0.261996 | Train accuracy: 0.648000 | Test accuracy: 0.690000
Accuracy of 4 qubit model with 5 layer: 0.69
CPU times: user 4min 40s, sys: 129 ms, total: 4min 40s
Wall time: 5min 3s
